In [1]:
# ═══ eICU-CRD CBC BIOMARKER ANALYSIS ═══
# Replication of MIMIC-IV analysis on eICU dataset

import duckdb
import pandas as pd
import numpy as np
from pathlib import Path
from itertools import combinations
from scipy.stats import pointbiserialr
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# ─── 1) DATABASE SETUP ─────────────────────────────────────────────────────
print("=== SETTING UP eICU DATABASE CONNECTION ===")

con = duckdb.connect()

# Set your data directory path
DATA_DIR = Path("/Users/dimitri/Desktop/indi/data/eicu-collaborative-research-database-2.0")
p = lambda f: str(DATA_DIR / f)

print(f"Data directory: {DATA_DIR}")

# ─── 2) CORE VIEWS SETUP ───────────────────────────────────────────────────
print("\n1. Creating core database views...")

# Patient view - includes mortality information
con.execute(f"""
CREATE OR REPLACE VIEW patients AS
SELECT 
    patientunitstayid,
    patienthealthsystemstayid,
    gender,
    age,
    unitdischargeoffset / 60.0 AS los_hours,
    hospitaldischargestatus,
    CASE 
        WHEN hospitaldischargestatus = 'Expired' THEN 1 
        ELSE 0 
    END AS hospital_mortality,
    -- For 30-day mortality, we'll use hospital mortality as proxy since eICU doesn't track post-discharge
    CASE 
        WHEN hospitaldischargestatus = 'Expired' THEN 1 
        ELSE 0 
    END AS mort_30d
FROM read_csv_auto('{p('patient.csv.gz')}')
WHERE age != '> 89'  -- Exclude de-identified elderly
  AND TRY_CAST(age AS INTEGER) >= 18  -- Adults only
""")

# Diagnoses view
con.execute(f"""
CREATE OR REPLACE VIEW diagnoses AS
SELECT 
    patientunitstayid,
    diagnosisstring,
    icd9code,
    diagnosisoffset
FROM read_csv_auto('{p('diagnosis.csv.gz')}')
""")

print("✓ Core views created")

# ─── 3) CBC LAB MAPPING ────────────────────────────────────────────────────
print("\n2. Mapping eICU lab names to CBC parameters...")

# First, let's explore what CBC-related labs are available
lab_exploration = con.execute(f"""
SELECT DISTINCT labname, COUNT(*) as count
FROM read_csv_auto('{p('lab.csv.gz')}')
WHERE labname ILIKE '%hgb%' 
   OR labname ILIKE '%hemoglobin%'
   OR labname ILIKE '%hematocrit%' 
   OR labname ILIKE '%hct%'
   OR labname ILIKE '%wbc%'
   OR labname ILIKE '%white%'
   OR labname ILIKE '%rbc%'
   OR labname ILIKE '%red%'
   OR labname ILIKE '%platelet%'
   OR labname ILIKE '%plt%'
   OR labname ILIKE '%mcv%'
   OR labname ILIKE '%mch%'
   OR labname ILIKE '%mchc%'
   OR labname ILIKE '%rdw%'
   OR labname ILIKE '%neutrophil%'
   OR labname ILIKE '%lymphocyte%'
   OR labname ILIKE '%monocyte%'
GROUP BY labname
ORDER BY count DESC
LIMIT 50
""").df()

print("Found CBC-related lab names:")
print(lab_exploration.head(20))

# ─── 4) CBC DATA EXTRACTION ────────────────────────────────────────────────
print("\n3. Extracting CBC data from lab table...")

# Create CBC mapping based on eICU lab names
con.execute(f"""
CREATE OR REPLACE TABLE enhanced_cbc AS
WITH first_labs AS (
    SELECT 
        l.patientunitstayid,
        l.labname,
        l.labresult,
        l.labresultoffset,
        ROW_NUMBER() OVER (PARTITION BY l.patientunitstayid, l.labname 
                          ORDER BY l.labresultoffset) as rn
    FROM read_csv_auto('{p('lab.csv.gz')}') l
    WHERE l.labresultoffset >= 0  -- After ICU admission
      AND l.labresultoffset <= 1440  -- Within first 24 hours (in minutes)
      AND TRY_CAST(l.labresult AS DOUBLE) IS NOT NULL  -- Valid numeric results
)
SELECT 
    p.patientunitstayid,
    p.los_hours,
    p.mort_30d,
    CASE WHEN p.los_hours > 72 THEN 1 ELSE 0 END AS prolonged_stay,
    -- CBC values using eICU lab names
    MAX(CASE WHEN labname = 'Hgb' THEN TRY_CAST(labresult AS DOUBLE) END) AS hb,
    MAX(CASE WHEN labname = 'Hct' THEN TRY_CAST(labresult AS DOUBLE) END) AS hct,
    MAX(CASE WHEN labname = 'MCV' THEN TRY_CAST(labresult AS DOUBLE) END) AS mcv,
    MAX(CASE WHEN labname = 'MCH' THEN TRY_CAST(labresult AS DOUBLE) END) AS mch,
    MAX(CASE WHEN labname = 'MCHC' THEN TRY_CAST(labresult AS DOUBLE) END) AS mchc,
    MAX(CASE WHEN labname = 'RDW' THEN TRY_CAST(labresult AS DOUBLE) END) AS rdw,
    MAX(CASE WHEN labname = 'RBC' THEN TRY_CAST(labresult AS DOUBLE) END) AS rbc,
    MAX(CASE WHEN labname = 'platelets x 1000' THEN TRY_CAST(labresult AS DOUBLE) END) AS platelet,
    MAX(CASE WHEN labname = 'WBC x 1000' THEN TRY_CAST(labresult AS DOUBLE) END) AS wbc,
    MAX(CASE WHEN labname = '-polys' THEN TRY_CAST(labresult AS DOUBLE) END) AS neutrophils,
    MAX(CASE WHEN labname = '-lymphs' THEN TRY_CAST(labresult AS DOUBLE) END) AS lymphocytes,
    MAX(CASE WHEN labname = '-monos' THEN TRY_CAST(labresult AS DOUBLE) END) AS monocytes
FROM patients p
LEFT JOIN first_labs l ON p.patientunitstayid = l.patientunitstayid AND l.rn = 1
GROUP BY p.patientunitstayid, p.los_hours, p.mort_30d
""")

print("✓ CBC data extracted")

# ─── 5) LOAD INTO PANDAS ───────────────────────────────────────────────────
print("\n4. Loading data into pandas...")

df_main = con.execute("""
    SELECT *
    FROM enhanced_cbc
    WHERE ((hb IS NULL)::INT + (hct IS NULL)::INT + (wbc IS NULL)::INT + 
           (platelet IS NULL)::INT + (rbc IS NULL)::INT) <= 2
""").df()

print(f"✓ Loaded {len(df_main)} ICU stays")
print(f"  30-day mortality rate: {df_main['mort_30d'].mean():.1%}")
print(f"  Prolonged stay rate: {df_main['prolonged_stay'].mean():.1%}")

# ─── 6) RATIO BIOMARKER CREATION ───────────────────────────────────────────
print("\n5. Creating ratio biomarkers...")

def create_ratio_biomarkers(df, base_params):
    """Create systematic ratio combinations from CBC parameters"""
    ratio_df = df.copy()
    
    # 2-parameter ratios (all combinations)
    for param1, param2 in combinations(base_params, 2):
        if param1 in df.columns and param2 in df.columns:
            # Forward ratio
            ratio_df[f'{param1}_to_{param2}'] = df[param1] / (df[param2] + 1e-8)
            # Reverse ratio  
            ratio_df[f'{param2}_to_{param1}'] = df[param2] / (df[param1] + 1e-8)
    
    # Known inflammatory indices
    if all(col in df.columns for col in ['neutrophils', 'lymphocytes']):
        ratio_df['nlr'] = df['neutrophils'] / (df['lymphocytes'] + 1e-8)
    
    if all(col in df.columns for col in ['platelet', 'lymphocytes']):
        ratio_df['plr'] = df['platelet'] / (df['lymphocytes'] + 1e-8)
    
    if all(col in df.columns for col in ['monocytes', 'lymphocytes']):
        ratio_df['mlr'] = df['monocytes'] / (df['lymphocytes'] + 1e-8)
    
    # Novel multi-parameter indices
    if all(col in df.columns for col in ['neutrophils', 'monocytes', 'lymphocytes']):
        ratio_df['siri'] = (df['neutrophils'] * df['monocytes']) / (df['lymphocytes'] + 1e-8)
    
    if all(col in df.columns for col in ['neutrophils', 'platelet', 'lymphocytes']):
        ratio_df['sii'] = (df['neutrophils'] * df['platelet']) / (df['lymphocytes'] + 1e-8)
    
    if all(col in df.columns for col in ['hb', 'rdw']):
        ratio_df['hrr'] = df['hb'] / (df['rdw'] + 1e-8)
    
    if all(col in df.columns for col in ['wbc', 'hb']):
        ratio_df['whr'] = df['wbc'] / (df['hb'] + 1e-8)
    
    return ratio_df

# Define base CBC parameters
base_cbc_params = ['hb', 'hct', 'mcv', 'mch', 'mchc', 'rdw', 'rbc', 
                   'platelet', 'wbc', 'neutrophils', 'lymphocytes', 'monocytes']

# Create ratio biomarkers
df_with_ratios = create_ratio_biomarkers(df_main, base_cbc_params)

# Get all ratio columns
ratio_cols = [col for col in df_with_ratios.columns 
              if ('_to_' in col or col in ['nlr', 'plr', 'mlr', 'siri', 'sii', 'hrr', 'whr'])]

print(f"✓ Created {len(ratio_cols)} ratio biomarkers")

# ─── 7) BIOMARKER EVALUATION ───────────────────────────────────────────────
print("\n6. Evaluating biomarkers for mortality prediction...")

def evaluate_biomarkers(df, biomarker_cols, outcome_col):
    """Evaluate biomarker performance for binary outcome"""
    results = []
    
    for biomarker in biomarker_cols:
        if biomarker not in df.columns:
            continue
            
        # Clean data
        clean_data = df[[biomarker, outcome_col]].dropna()
        clean_data = clean_data.replace([np.inf, -np.inf], np.nan).dropna()
        
        if len(clean_data) < 100 or clean_data[outcome_col].nunique() < 2:
            continue
            
        try:
            # Calculate performance metrics
            auc = roc_auc_score(clean_data[outcome_col], clean_data[biomarker])
            r_pb, p_pb = pointbiserialr(clean_data[outcome_col], clean_data[biomarker])
            
            # Calculate effect size (Cohen's d)
            group_0 = clean_data[clean_data[outcome_col] == 0][biomarker]
            group_1 = clean_data[clean_data[outcome_col] == 1][biomarker]
            
            mean_diff = group_1.mean() - group_0.mean()
            pooled_std = np.sqrt(((len(group_0)-1)*group_0.var() + 
                                 (len(group_1)-1)*group_1.var()) / 
                                (len(group_0)+len(group_1)-2))
            cohens_d = abs(mean_diff) / (pooled_std + 1e-8)
            
            results.append({
                'biomarker': biomarker,
                'auc': auc,
                'cohens_d': cohens_d,
                'r_pointbiserial': abs(r_pb),
                'p_pointbiserial': p_pb,
                'n_samples': len(clean_data)
            })
            
        except Exception as e:
            continue
    
    results_df = pd.DataFrame(results)
    results_df['composite_score'] = (
        0.4 * (results_df['auc'] - 0.5) / 0.5 +  # Normalized AUC
        0.3 * results_df['cohens_d'] / results_df['cohens_d'].max() +
        0.3 * results_df['r_pointbiserial'] / results_df['r_pointbiserial'].max()
    )
    
    return results_df.sort_values('composite_score', ascending=False)

# Include all features for evaluation
all_features = base_cbc_params + ratio_cols
mortality_results = evaluate_biomarkers(df_with_ratios, all_features, 'mort_30d')

# ─── 8) DISPLAY RESULTS ────────────────────────────────────────────────────
print("\n" + "="*80)
print("TOP 20 BIOMARKERS FOR MORTALITY IN eICU")
print("="*80)

# Show top 20 biomarkers
top_20 = mortality_results.head(20)[['biomarker', 'auc', 'cohens_d', 'composite_score']]
print(top_20.to_string(index=False))

# Check if RDW/MCHC is in top performers
rdw_mchc_rank = None
if 'rdw_to_mchc' in mortality_results['biomarker'].values:
    rdw_mchc_rank = mortality_results[mortality_results['biomarker'] == 'rdw_to_mchc'].index[0] + 1
    rdw_mchc_perf = mortality_results[mortality_results['biomarker'] == 'rdw_to_mchc'].iloc[0]
    
    print(f"\n" + "="*80)
    print(f"RDW/MCHC RATIO PERFORMANCE IN eICU")
    print("="*80)
    print(f"Rank: {rdw_mchc_rank} out of {len(mortality_results)}")
    print(f"AUC: {rdw_mchc_perf['auc']:.3f}")
    print(f"Cohen's d: {rdw_mchc_perf['cohens_d']:.3f}")
    print(f"Samples: {rdw_mchc_perf['n_samples']}")

# ─── 9) COMPARISON WITH MIMIC-IV ──────────────────────────────────────────
print("\n" + "="*80)
print("DATASET COMPARISON")
print("="*80)
print(f"eICU patients: {len(df_main)}")
print(f"eICU mortality rate: {df_main['mort_30d'].mean():.1%}")
print(f"eICU data completeness:")
for param in ['hb', 'hct', 'wbc', 'platelet', 'rdw', 'neutrophils']:
    if param in df_main.columns:
        completeness = df_main[param].notna().sum() / len(df_main) * 100
        print(f"  {param}: {completeness:.1f}%")

# ─── 10) SAVE RESULTS ─────────────────────────────────────────────────────
print("\n7. Saving results...")
mortality_results.to_csv('eicu_biomarker_results.csv', index=False)
df_with_ratios.to_csv('eicu_cbc_with_ratios.csv', index=False)
print("✓ Results saved")

print("\n" + "="*80)
print("✅ eICU ANALYSIS COMPLETE!")
print("="*80)

# ─── 11) DISEASE STRATIFICATION (BONUS) ────────────────────────────────────
print("\n8. Performing disease stratification analysis...")

# Create disease categories from diagnosis strings
con.execute(f"""
CREATE OR REPLACE TABLE disease_categories AS
WITH primary_diagnosis AS (
    SELECT 
        d.patientunitstayid,
        d.diagnosisstring,
        CASE 
            WHEN diagnosisstring ILIKE '%sepsis%' OR diagnosisstring ILIKE '%septic%' THEN 'Sepsis'
            WHEN diagnosisstring ILIKE '%pneumonia%' OR diagnosisstring ILIKE '%respiratory%' 
                 OR diagnosisstring ILIKE '%ARDS%' OR diagnosisstring ILIKE '%COPD%' THEN 'Respiratory'
            WHEN diagnosisstring ILIKE '%cardiac%' OR diagnosisstring ILIKE '%heart%' 
                 OR diagnosisstring ILIKE '%myocardial%' THEN 'Cardiovascular'
            WHEN diagnosisstring ILIKE '%stroke%' OR diagnosisstring ILIKE '%neuro%' 
                 OR diagnosisstring ILIKE '%brain%' THEN 'Neurological'
            WHEN diagnosisstring ILIKE '%trauma%' OR diagnosisstring ILIKE '%injury%' THEN 'Trauma'
            WHEN diagnosisstring ILIKE '%renal%' OR diagnosisstring ILIKE '%kidney%' THEN 'Renal'
            WHEN diagnosisstring ILIKE '%GI%' OR diagnosisstring ILIKE '%gastro%' 
                 OR diagnosisstring ILIKE '%bleeding%' THEN 'Gastrointestinal'
            ELSE 'Other'
        END AS disease_category,
        ROW_NUMBER() OVER (PARTITION BY d.patientunitstayid ORDER BY d.diagnosisoffset) as rn
    FROM diagnoses d
)
SELECT 
    patientunitstayid,
    disease_category
FROM primary_diagnosis
WHERE rn = 1
""")


# Merge with main data
df_with_disease = con.execute("""
    SELECT 
        c.*,
        COALESCE(d.disease_category, 'Other') as disease_category
    FROM enhanced_cbc c
    LEFT JOIN disease_categories d USING(patientunitstayid)
""").df()

# Analyze by disease
disease_summary = df_with_disease.groupby('disease_category').agg({
    'mort_30d': ['count', 'mean']
}).round(3)

print("\nDisease distribution in eICU:")
print(disease_summary)

print("\n✅ Full analysis complete!")

=== SETTING UP eICU DATABASE CONNECTION ===
Data directory: /Users/dimitri/Desktop/indi/data/eicu-collaborative-research-database-2.0

1. Creating core database views...
✓ Core views created

2. Mapping eICU lab names to CBC parameters...
Found CBC-related lab names:
                         labname    count
0                            Hgb  1298708
1                            Hct  1279704
2               platelets x 1000  1149488
3                     WBC x 1000  1134952
4                            RBC  1132361
5                            MCV  1105350
6                           MCHC  1104854
7                            MCH  1059865
8                            RDW  1048569
9                  Methemoglobin   101924
10             Carboxyhemoglobin   100040
11                 Oxyhemoglobin    35738
12                WBC's in urine    24453
13  WBC's in cerebrospinal fluid     3331
14           WBC's in body fluid     2022
15        WBC's in pleural fluid      456
16     WBC's in pe

NameError: name 'results_mort_ranked' is not defined